# Retrieving ADA-BTC historical data 

For this step, we are going to retrieve historical data of ADA/BTC from Binance API. To read more about Binance API visit https://binance-docs.github.io/apidocs/. We are using python-binance library to help us achieve our goal. Finally, we are transforming our datasets into a csv file. We will analyze our datasets in the csv file on the next step. To read more about python-binance library visit https://python-binance.readthedocs.io/en/latest/ .

In [20]:
# requirements
# pip install python-binance

# import libraries
import pandas as pd
import math
import os.path
import time
from binance import Client
from datetime import timedelta, datetime
from dateutil import parser

In [21]:
api_key = [YOUR_API_KEY]
api_secret = [YOUR_SECRET_KEY]

Binance API requires an interval parameter in minutes for klines datasets. We are interested in a one-hour interval. Additionally, for klines data, Binance API take a limit parameter to restrict the amount of returned data. By default, its limit is 500. We are going to set the limit to the maximum allowed limit, 1000.

In [22]:
binsizes = {'1h': 60}
batch_size = 1000
client = Client(api_key, api_secret) 

From https://binance-docs.github.io/apidocs/#kline-candlestick-data we can see that Binance API store the time value in timestamp in milliseconds format. We are going to transform the timestamp into Date Time format using pandas.to_datetime.

In [23]:
def minutes_of_new_data(symbol, kline_size, data, source):
    if len(data) > 0:  
        old = parser.parse(data["timestamp"].iloc[-1])
    elif source == "binance": 
        old = datetime.strptime('27 Sep 2017', '%d %b %Y')
    if source == "binance": 
        new = pd.to_datetime(client.get_klines(symbol=symbol, interval=kline_size)[-1][0], unit='ms')
    return old, new

We are going to retrieve historical data of the desired pairs using Client.get_historical_klines from python-binance library. Then, we export our datasets into csv files.

In [24]:
def get_all_binance(symbol, kline_size, save = False):
    filename = '%s-%s-data.csv' % (symbol, kline_size)
    if os.path.isfile(filename): 
        data_df = pd.read_csv(filename)
    else: 
        data_df = pd.DataFrame()
        
    oldest_point, newest_point = minutes_of_new_data(symbol, kline_size, data_df, source = "binance")
    delta_min = (newest_point - oldest_point).total_seconds()/60
    available_data = math.ceil(delta_min/binsizes[kline_size])
    
    if oldest_point == datetime.strptime('27 Sep 2017', '%d %b %Y'): 
        print('Downloading all available %s data for %s. Be patient..!' % (kline_size, symbol))
    else: 
        print('Downloading %d minutes of new data available for %s, i.e. %d instances of %s data.' % (delta_min, symbol, available_data, kline_size))
        
    klines = client.get_historical_klines(symbol, kline_size, oldest_point.strftime("%d %b %Y %H:%M:%S"), newest_point.strftime("%d %b %Y %H:%M:%S"))
    data = pd.DataFrame(klines, columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore' ])
    data['timestamp'] = pd.to_datetime(data['timestamp'], unit='ms')
    if len(data_df) > 0:
        temp_df = pd.DataFrame(data)
        data_df = data_df.append(temp_df)
    else: 
        data_df = data
    data_df.set_index('timestamp', inplace=True)
    if save: 
        data_df.to_csv(filename)
    print('All caught up..!')
    return data_df

In [14]:
# Dataset for pair ADA/TUSD. I misread TUSD for USD. We ended up not needing this dataset. We can ignore this part.
data = get_all_binance('ADATUSD','1h',save =True)

All caught up..!


In [16]:
data

,open,high,low,close,volume,close_time,quote_av,trades,tb_base_av,tb_quote_av,ignore
timestamp,,,,,,,,,,,
2018-12-19 04:00:00,0.03704000,0.03704000,0.03490000,0.03490000,615087.10000000,1545195599999,22093.09266900,21,531465.60000000,19119.08257400,0
2018-12-19 05:00:00,0.03508000,0.03543000,0.03508000,0.03533000,231344.10000000,1545199199999,8147.08605500,20,69184.00000000,2447.16045300,0
2018-12-19 06:00:00,0.03555000,0.03561000,0.03508000,0.03522000,24392.70000000,1545202799999,862.50245900,13,9244.70000000,328.58153500,0
2018-12-19 07:00:00,0.03540000,0.03540000,0.03468000,0.03474000,174010.80000000,1545206399999,6080.12837300,19,25119.70000000,887.47078600,0
2018-12-19 08:00:00,0.03498000,0.03510000,0.03485000,0.03510000,63295.30000000,1545209999999,2211.63010000,12,48280.60000000,1687.67221700,0
...,...,...,...,...,...,...,...,...,...,...,...
2021-08-19 19:00:00,2.30210000,2.39990000,2.28090000,2.38620000,115648.28000000,1629403199999,271371.75129800,451,58993.21000000,139020.94313500,0
2021-08-19 20:00:00,2.40000000,2.43760000,2.31490000,2.35590000,83598.91000000,1629406799999,200762.55824900,443,59334.92000000,142787.60839500,0
2021-08-19 21:00:00,2.37000000,2.39230000,2.33830000,2.38580000,7420.91000000,1629410399999,17483.58080000,91,3548.21000000,8376.33461700,0


We are retrieving historical data for ADA/BTC with interval of one-hour and save them into a csv file.

In [17]:
data2 = get_all_binance('ADABTC','1h',save =True)

All caught up..!


In [19]:
data2

,open,high,low,close,volume,close_time,quote_av,trades,tb_base_av,tb_quote_av,ignore
timestamp,,,,,,,,,,,
2017-11-30 12:00:00,0.00000900,0.00008900,0.00000900,0.00001230,5468798.00000000,1512046799999,70.90734001,758,2932866.00000000,38.86051313,14884618.46300700
2017-11-30 13:00:00,0.00001230,0.00001231,0.00001107,0.00001135,4706220.00000000,1512050399999,54.37609725,858,2027600.00000000,23.41386248,24471779.53211700
2017-11-30 14:00:00,0.00001135,0.00001270,0.00001134,0.00001221,7824209.00000000,1512053999999,94.06347795,796,4784161.00000000,57.40436315,28942850.95876700
2017-11-30 15:00:00,0.00001221,0.00001222,0.00001146,0.00001189,6190482.00000000,1512057599999,73.36461964,690,4478927.00000000,53.06816956,32239890.52905500
2017-11-30 16:00:00,0.00001194,0.00001271,0.00001169,0.00001212,8185846.00000000,1512061199999,98.83704689,822,4990925.00000000,60.38901944,34165345.01920200
...,...,...,...,...,...,...,...,...,...,...,...
2021-08-19 20:00:00,0.00005129,0.00005229,0.00004987,0.00005084,12482636.00000000,1629406799999,637.43231733,32369,6215172.00000000,317.61409453,0
2021-08-19 21:00:00,0.00005083,0.00005135,0.00005030,0.00005135,4449271.00000000,1629410399999,225.48885975,10336,1978141.00000000,100.27308376,0
2021-08-19 22:00:00,0.00005133,0.00005134,0.00005067,0.00005109,3923286.00000000,1629413999999,200.10138636,7582,2176253.00000000,111.00520103,0


We are going to analyze our datasets on the next step.